<a href="https://colab.research.google.com/github/YasraNafees/Tweet-analysis/blob/main/tweet_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# STEP 1: SETUP KAGGLE API IN COLAB
!pip install -q kaggle
from google.colab import files
import os

# Upload your kaggle.json (from Windows: C:\Users\[YourUser]\.kaggle\)
print("Please upload your kaggle.json file")
uploaded = files.upload()

# Configure environment
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Verify setup
!kaggle config view

# STEP 2: DOWNLOAD SENTIMENT140 DATASET
!kaggle datasets download -d kazanova/sentiment140 --force

# STEP 3: UNZIP AND LOAD DATA
import zipfile
import pandas as pd

# Unzip (handles Windows/Unix paths automatically in Colab)
with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall('sentiment_data')

# Load dataset (with proper column names)
cols = ["target", "ids", "date", "flag", "user", "text"]
df = pd.read_csv('sentiment_data/training.1600000.processed.noemoticon.csv',
                 encoding='latin-1',
                 header=None,
                 names=cols)

# STEP 4: QUICK DATA EXPLORATION
print(f"Dataset shape: {df.shape}")
print("\nFirst 3 tweets:")
for i, row in df.head(3).iterrows():
    print(f"\nTarget: {row['target']} (0=negative, 4=positive)")
    print(f"Text: {row['text']}")

# STEP 5: SAVE TO GOOGLE DRIVE (OPTIONAL)
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/My Drive/sentiment140_cleaned.csv', index=False)
print("\nData saved to Google Drive!")

Please upload your kaggle.json file


Saving kaggle.json to kaggle.json
Configuration values from /root/.kaggle
- username: yasranafees
- path: None
- proxy: None
- competition: None
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 945MB/s]
Dataset shape: (1600000, 6)

First 3 tweets:

Target: 0 (0=negative, 4=positive)
Text: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D

Target: 0 (0=negative, 4=positive)
Text: is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!

Target: 0 (0=negative, 4=positive)
Text: @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Data saved to Google Drive!


In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Printing the stop words

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

# Data Processing

In [ ]:

# Load your data
# Specify column names as the CSV does not have a header
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
df = pd.read_csv('/content/sentiment_data/training.1600000.processed.noemoticon.csv',
                 encoding='latin-1',
                 header=None, # Indicate that the file has no header
                 names=column_names) # Assign the specified column names


In [ ]:
# checking the number of rows and columns
df.shape

(1600000, 6)

In [ ]:
# printing the first 5 rows
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the missing values
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# destribution of target column
df['target'].value_counts()

,count
target,
0,800000
4,800000


# Convert the target 4 into 1

In [ ]:
df.replace({'target':{4:1}},inplace=True)

In [ ]:
# checking the distribution of target column
df['target'].value_counts()

,count
target,
0,800000
1,800000


 0 means Negative Tweets

 1 means Positive Tweets

**Stemming**

Stemming is the process of reducing a word to its Root word

Example : actor, actoress, acting =act

In [ ]:
port_stem=PorterStemmer()

In [ ]:
# Example column names: 'text' and 'target'
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)     # Remove mentions
    text = re.sub(r"#\w+", "", text)     # Remove hashtags
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuations
    return text

# Now the 'text' column exists, apply the cleaning function
df['clean_text'] = df['text'].astype(str).apply(clean_text)

In [ ]:
df.head(90)

,target,ids,date,flag,user,text,clean_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww thats a bummer you shoulda got david ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i he...
...,...,...,...,...,...,...,...
85,0,1467834400,Mon Apr 06 22:25:56 PDT 2009,NO_QUERY,calihonda2001,"had a blast at the Getty Villa, but hates that...",had a blast at the getty villa but hates that ...
86,0,1467834817,Mon Apr 06 22:26:02 PDT 2009,NO_QUERY,djwayneski,@msdrama hey missed ya at the meeting sup mama,hey missed ya at the meeting sup mama
87,0,1467835085,Mon Apr 06 22:26:06 PDT 2009,NO_QUERY,Ceejison,My tummy hurts. I wonder if the hypnosis has ...,my tummy hurts i wonder if the hypnosis has a...
88,0,1467835198,Mon Apr 06 22:26:08 PDT 2009,NO_QUERY,ItsBrigittaYo,why is it always the fat ones?!,why is it always the fat ones


In [ ]:
print(df['clean_text'])

0             awww thats a bummer  you shoulda got david ...
1          is upset that he cant update his facebook by t...
2           i dived many times for the ball managed to sa...
3            my whole body feels itchy and like its on fire 
4           no its not behaving at all im mad why am i he...
                                 ...                        
1599995    just woke up having no school is the best feel...
1599996    thewdbcom  very cool to hear old walt intervie...
1599997    are you ready for your mojo makeover ask me fo...
1599998    happy 38th birthday to my boo of alll time tup...
1599999                                           happy     
Name: clean_text, Length: 1600000, dtype: object


In [ ]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# separate the data and label
X=df['text'].values
Y=df['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 ... 'Are you ready for your MoJo Makeover? Ask me for details '
 'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '
 'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H ']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


# Spliting the data to test train split

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
# Fit and transform the cleaned text data
X_vectorized = vectorizer.fit_transform(df['clean_text'])

# Get the target variable
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


In [ ]:
print(X_test.shape)

(320000, 5000)


In [ ]:
print(X_train.shape)

(1280000, 5000)


# Training the Machine Learning Model

Logistic Regression

In [ ]:
# Logistic Regression
model = LogisticRegression(max_iter=1000)
# Fit the model using the vectorized training data
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Predictions
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, train_pred))
print("Testing Accuracy:", accuracy_score(y_test, test_pred))
print(classification_report(y_test, test_pred))

Training Accuracy: 0.7970921875
Testing Accuracy: 0.794984375
              precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.79    320000
   macro avg       0.80      0.79      0.79    320000
weighted avg       0.80      0.79      0.79    320000



# accuracy Score is 79.7%

Saving the model

In [ ]:
import pickle
import joblib

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

# using the save model for future prediction

In [ ]:
# loading the save model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [ ]:
# loading the save model
loaded_model=pickle.load(open('trained_model.sav','rb'))

# Access the element by its positional index (0-based)
X_new=X_test[200]
print(y_test.iloc[200]) # Use iloc for positional indexing on y_test
prediction=loaded_model.predict(X_new)
print(prediction)
if prediction[0]==0:
  print('Negative tweets')
else:
  print('Positive tweets')

0
[0]
Negative tweets


In [ ]:
# loading the save model
loaded_model=pickle.load(open('trained_model.sav','rb'))

# Access the element by its positional index (0-based)
X_new=X_test[80]
print(y_test.iloc[80]) # Use iloc for positional indexing on y_test
prediction=loaded_model.predict(X_new)
print(prediction)
if prediction[0]==0:
  print('Negative tweets')
else:
  print('Positive tweets')

1
[1]
Positive tweets


In [ ]:
# loading the save model
loaded_model=pickle.load(open('trained_model.sav','rb'))

# Access the element by its positional index (0-based)
X_new=X_test[87]
print(y_test.iloc[87]) # Use iloc for positional indexing on y_test
prediction=loaded_model.predict(X_new)
print(prediction)
if prediction[0]==0:
  print('Negative tweets')
else:
  print('Positive tweets')

1
[1]
Positive tweets


In [ ]:
!pip install wordninja


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=a7cf418e0e73322d0f8a14702cef3ac22034f30599b013b57cb74fecf51bd383
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [ ]:
def segment_text(text):
    # Simple whitespace-based tokenization
    return ' '.join(text.split())

new_text = ["I love this!", "This is terrible."]
new_text_clean = [segment_text(t.lower()) for t in new_text]
new_vec = vectorizer.transform(new_text_clean)
print(model.predict(new_vec))

[1 0]


1 means positive

0 means Negative

In [ ]:
print(model.classes_)  # This will show what classes your model predicts

[0 1]


In [ ]:
print(model.predict_proba(new_vec))

[[0.0206105  0.9793895 ]
 [0.95356258 0.04643742]]


In [ ]:
for text, pred in zip(new_text, model.predict(new_vec)):
    print(f"'{text}' → {'Positive' if pred == 1 else 'Negative'}")

'I love this!' → Positive
'This is terrible.' → Negative


In [ ]:
def predict_sentiment(text):
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    model = joblib.load('trained_model.sav')

    text_vector = vectorizer.transform([text])
    prediction = model.predict(text_vector)

    return prediction[0]


In [ ]:
predict_sentiment("I love this product!")  # Output: 'Positive' or similar


np.int64(1)